# Create files for heatmapping

## Generate bigwig files for each replicate 

In [2]:
%%bash 
cat generate_activity_bigwigs.slrm
sbatch generate_activity_bigwigs.slrm

#!/bin/bash
#SBATCH --mail-user=tyler.j.hansen@vanderbilt.edu
#SBATCH --mail-type=ALL
#SBATCH --cpus-per-task=12
#SBATCH --time=24:00:00
#SBATCH --mem=128G
#######################################################
#Tyler Hansen | 12.15.2021
#This script generates signal files for my ATAC-STARR replicates. 

#software
module load Anaconda3
source activate jupyter #see repo for required software and conda clone. 

#Paths
BAM_DIR='/data/hodges_lab/ATAC-STARR_B-cells/data/ATAC-STARR/bams/individual_replicates'
OUT_DIR='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs'

for i in GM12878inGM12878 GM12878inLCL8664 LCL8664inGM12878 LCL8664inLCL8664
do
    for j in Rep1 Rep2 Rep3
    do
        python3 generate_ATAC-STARR_bigwig.py -d ${BAM_DIR}/${i}_DNA_${j}.filtered.pos-sorted.bam \
            -r ${BAM_DIR}/${i}_RNA_${j}.filtered.pos-sorted.bam -n ${i}_${j} \
            -o $OUT_DIR -bs 10 -p 12
    done
doneSubmitted batch job 36056053


In [4]:
%%bash
#The script above uses a python script I wrote. Here is its functionallity: 
python3 generate_ATAC-STARR_bigwig.py --help

usage: generate_ATAC-STARR_bigwig.py [-h] [-d DNA_BAM] [-r RNA_BAM] [-n NAME]
                                     [-o OUT_DIR] [-bs BIN_SIZE] [-p THREADS]

Generate ATAC-STARR bigWig

required arguments:
  -d DNA_BAM, --DNA_bam DNA_BAM
                        reisolated plasmid DNA bam file
  -r RNA_BAM, --RNA_bam RNA_BAM
                        reporter RNA bam file
  -n NAME, --name NAME  name of file prefix for output files
  -o OUT_DIR, --out_dir OUT_DIR
                        output directory

optional arguments:
  -h, --help            show this help message and exit
  -bs BIN_SIZE, --bin_size BIN_SIZE
                        binsize parameter for the bigwig generation (default:
                        50bp)
  -p THREADS, --threads THREADS
                        number of threads (default: 1)


## LiftOver rheMac10 bigwigs to hg38

In [14]:
%%bash
#first remove extra chr names from bigwig. These do no handle well with crossmap

DIR='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs'

#stop on any error
set -ue

for name in LCL8664inLCL8664 LCL8664inGM12878
do
    for rep in Rep1 Rep2 Rep3
    do
        for file in RNA-to-DNA_log2 RNA-to-DNA_subtract DNA-coverage RNA-coverage
        do
            #VARIABLES
            A=${DIR}/${name}_${rep}_${file}.bw
            B=${DIR}/${name}_${rep}_${file}.bedGraph
            C=${DIR}/${name}_${rep}_${file}.filtered.bedGraph
            D=${DIR}/${name}_${rep}_${file}.filtered.bw

            #convert to bedgraph
            bigWigToBedGraph $A $B

            #filter to remove anything not normal
            awk '$1 !~ /_/' $B > $C

            #convert back to bigwig
            bedGraphToBigWig $C /home/hansetj1/chrom.sizes/rheMac10.chrom.sizes $D
        done
    done
done

In [17]:
%%bash
cat crossmap_bigwigs.slrm
sbatch crossmap_bigwigs.slrm

#!/bin/bash
#SBATCH --mail-user=tyler.j.hansen@vanderbilt.edu
#SBATCH --mail-type=ALL
#SBATCH --cpus-per-task=1
#SBATCH --time=72:00:00
#SBATCH --mem=32G
#######################################################
#Tyler Hansen | 12.15.2021
#This script generates signal files for my ATAC-STARR replicates. 

#software
module load Anaconda3
source activate jupyter 

#stop on any error and write out commands as they execute
set -ue

#Variables
CHAIN='/data/hodges_lab/bin/map.chain/rheMac10ToHg38.over.chain.gz'
DIR='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs'

#do for comparison files first and then run on individual files
for name in LCL8664inGM12878 LCL8664inLCL8664
do 
    for rep in Rep1 Rep2 Rep3
    do
        for comparison in log2 subtract
        do
            #set file variables
            IN=${DIR}/${name}_${rep}_RNA-to-DNA_${comparison}.filtered.bw
            OUT=${DIR}/${name}_${rep}_RNA-to-DNA_${comparison}.filtered.hg38-crossmap #leav

## Add to config.ini

In [1]:
#function
append_section_to_ini <- function(ini_file, new_section) {
    if (is.list(new_section)) {
        ini <- ini::read.ini(ini_file)
        ini <- c(ini,new_section)
        ini::write.ini(x=ini, filepath=ini_file)
    } else {
        print('new_section is not a list')
    }
}

#list of key-value pairs
ACTIVITY_BW <- list(dir='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs', 
                      HH_merged='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/GM12878inGM12878_merged_RNA-to-DNA_log2.bw', 
                      HM_merged='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/GM12878inLCL8664_merged_RNA-to-DNA_log2.bw', 
                      MH_merged='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/LCL8664inGM12878_merged_RNA-to-DNA_log2.filtered.hg38-crossmap.bw',
                      MM_merged='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/LCL8664inLCL8664_merged_RNA-to-DNA_log2.filtered.hg38-crossmap.bw', 
                      HH_R1='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/GM12878inGM12878_Rep1_RNA-to-DNA_log2.bw',
                      HH_R2='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/GM12878inGM12878_Rep2_RNA-to-DNA_log2.bw',
                      HH_R3='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/GM12878inGM12878_Rep3_RNA-to-DNA_log2.bw',
                      HM_R1='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/GM12878inLCL8664_Rep1_RNA-to-DNA_log2.bw',
                      HM_R2='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/GM12878inLCL8664_Rep2_RNA-to-DNA_log2.bw',
                      HM_R3='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/GM12878inLCL8664_Rep3_RNA-to-DNA_log2.bw',
                      MH_R1='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/LCL8664inGM12878_Rep1_RNA-to-DNA_log2.filtered.hg38-crossmap.bw',
                      MH_R2='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/LCL8664inGM12878_Rep2_RNA-to-DNA_log2.filtered.hg38-crossmap.bw',
                      MH_R3='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/LCL8664inGM12878_Rep3_RNA-to-DNA_log2.filtered.hg38-crossmap.bw',
                      MM_R1='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/LCL8664inLCL8664_Rep1_RNA-to-DNA_log2.filtered.hg38-crossmap.bw',
                      MM_R2='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/LCL8664inLCL8664_Rep2_RNA-to-DNA_log2.filtered.hg38-crossmap.bw',
                      MM_R3='/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/activity_bigwigs/LCL8664inLCL8664_Rep3_RNA-to-DNA_log2.filtered.hg38-crossmap.bw')

#list of section, resulting in list of list.
new_section <- list(ACTIVITY_BW=ACTIVITY_BW)

#write ini
append_section_to_ini("../config.ini", new_section)